In [1]:
from boto3.session import Session
import boto3
import botocore

In [2]:
from data_management_test.loader import load_credential

In [3]:
ACCESS_KEY = load_credential("AWS_ACCESS_KEY_ID","")
SECRET_ACCESS_KEY = load_credential("AWS_SECRET_ACCESS_KEY","")

In [ ]:
class S3Manager():
    def __init__(self):
        self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
        self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
        self.s3 =  boto3.resource('s3',
                                aws_access_key_id=ACCESS_KEY,
                                aws_secret_access_key=SECRET_ACCESS_KEY)

    def get_file_lists(self, bucket_name, s3_folder):
        lists = []
        for s3_file in self.s3.Bucket(bucket_name).objects.all():
            if s3_file.key.split('/')[0] == s3_folder:
                lists.append(s3_file.key.split('/')[1])
        return lists[5:9]
    
    def download_s3_files(self, bucket_name, s3_folder, local_path):
        # local_path 는 jupyter를 실행한 폴더 기준
        # 폴더 경로는 \\로 끝나야 함 (for Window)
        file_list = self.get_file_lists(bucket_name, s3_folder)
        print(file_list)
        for file in file_list:
            KEY = s3_folder + '/' + file
            print(KEY)
            try:
                self.s3.Bucket(bucket_name).download_file(KEY, local_path + file)
                print('downloaded')
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] == "404":
                    print("The object does not exist.")
                else:
                    print('----')
                    raise